In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Federated Project/FL-project

/content/drive/MyDrive/Federated Project/FL-project


In [3]:
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from main import *
import argparse
from centralized import Centralized
from client import *
import wandb

In [5]:
class Args():
  def __init__(self, dataset, model, bs, hnm, num_epochs):
    self.dataset = dataset
    self.model = model 
    self.bs = bs
    self.hnm = hnm
    self.num_epochs = num_epochs

args = Args('iddaCB', model = 'deeplabv3_mobilenetv2', bs = 16, hnm = False, num_epochs = 5)

In [6]:
trainAloneDataset, testAloneDatasets = get_datasets(args)
ds = trainAloneDataset[0]

In [7]:
#istanziamo il modello 
model = model_init(args)

In [8]:
sweep_config = {
   'method': 'random',
}



In [9]:
metric = { 
    'name' : 'miou',
    'goal' : 'minimize'
}

sweep_config['metric'] = metric

In [10]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
}

sweep_config['parameters'] = parameters_dict




In [11]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })


parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [12]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'miou'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'epochs': {'value': 1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [13]:
sweep_id = wandb.sweep(sweep_config, project="Task1")

Create sweep with ID: r3qqfeqf
Sweep URL: https://wandb.ai/mldlpolito/Task1/sweeps/r3qqfeqf


In [15]:
centralized = Centralized(args,ds,model)

wandb.agent(sweep_id, centralized.train2(), count=5)


params freezed
epoch 1 / 5, step 1 / 37, loss = 2.603
epoch 2 / 5, step 1 / 37, loss = 2.684
epoch 3 / 5, step 1 / 37, loss = 2.640
epoch 4 / 5, step 1 / 37, loss = 2.681
epoch 5 / 5, step 1 / 37, loss = 2.687
Finish training
Model saved


batch loss,▁█▄▇█
epoch,▁▃▅▆█
loss,▁█▄▇█
batch loss,2.68743
epoch,4
loss,0.07263


wandb: Agent Starting Run: vadg1oo7 with config:
	batch_size: 48
	epochs: 1
	learning_rate: 0.09406189926964326
	optimizer: adam
wandb: Agent Starting Run: pv936bcm with config:
	batch_size: 56
	epochs: 1
	learning_rate: 0.007211040976530059
	optimizer: adam
wandb: Agent Starting Run: z14t8ccb with config:
	batch_size: 80
	epochs: 1
	learning_rate: 0.024648728838562596
	optimizer: sgd
wandb: Agent Starting Run: fjftysir with config:
	batch_size: 88
	epochs: 1
	learning_rate: 0.06214262547932939
	optimizer: adam
wandb: Agent Starting Run: n0u0t5dk with config:
	batch_size: 40
	epochs: 1
	learning_rate: 0.06367305300072744
	optimizer: adam


2023-04-29 16:32:58,499 - wandb.wandb_agent - ERROR - Detected 5 failed runs in a row, shutting down.
